# Dataset  generation using Webots simulator

## Data log
* db_v1_20181019.pickle: 5 X 8 = __40__
* db_aug_v1_20181019.pickle: 5 X 8 X 20 X 3 = __2400__ -- augment by rotation (yaw, pitch)
* db_v2_20181019.pickle: 7 X 8 = __56__ -- add bus and truck data
* db_aug_v2_20181019.pickle: 7 X 8 X 20 X 3 = __3360__ -- augment by rotation (yaw, pitch)
* db_aug_v2_v_050_20181019.pickle: 7 X 8 X 20 X 3 = __3360__ -- voxel filter size: 0.05
* db_aug_v2_v_100_20181019.pickle: 7 X 8 X 20 X 3 = __3360__ -- voxel filter size: 0.10
* db_aug_v2_v_200_20181019.pickle: 7 X 8 X 20 X 3 = __3360__ -- voxel filter size: 0.20
* db_aug_v2_v_500_20181019.pickle: 7 X 8 X 20 X 3 = __3360__ -- voxel filter size: 0.50
* db_v3_20181031.pickle: 8 X 8 = __64__ -- add pedestrian
* db_v4_20181031.pickle: 10 X 8 = __80__ -- add bikes

## List of Instances 

#### Car
* world_BmwX5Simple
* world_ToyotaPriusSimple
* world_CitroenCZeroSimple
* world_LincolnMKZSimple
* world_RangeRoverSportSVRSimple
  - Vehicle1: 5.5 / 0.3 / -7
  - Vehicle1: 5.5 / 0.3 / 0
  - Vehicle1: 5.5 / 0.3 / 7
  - Vehicle1: 0 / 0.3 / -7
  - Vehicle1: 0 / 0.3 / 7
  - Vehicle1: -5.5 / 0.3 / -7
  - Vehicle1: -5.5 / 0.3 / 0
  - Vehicle1: -5.5 / 0.3 / 7

--> (5) X (8)

###### Webot data
![image](../image/vehicle.png)

###### Point cloud data
![image](../image/pointcloud.png)

#### Truck
Updated in version 2
* world_TruckSimple
* world_BusSimple

###### Webot data
![image](../image/bus.png)

###### Point cloud data
![image](../image/truck.png)



#### Pedestrian
* world_Pedestrian (TODO)

#### Bike
* world_ScooterSimple (TODO)
* world_MotorbikeSimple (TODO)

In [1]:
# localization data parser
import os, sys

import pcl

import numpy as np

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

sys.path.append( os.path.abspath("../scripts") )
from utils import *

%matplotlib qt

import pickle

import cv2

In [4]:
""" Bike """
root_path = os.path.abspath('../webots/bikes/')

bikes_list = []
bikes_list.append('world_MotorbikeSimple')
bikes_list.append('world_ScooterSimple')

idx = 1

file_path = os.path.join(root_path, bikes_list[idx])
image_path = os.path.join( os.path.abspath('../image') ,bikes_list[idx] + '.png')

In [6]:
point_cloud_path = os.path.join(file_path, 'PointCloud_0.bin')

file_p_list = FnParsePointCloudFile( point_cloud_path )

pcl_cloud = GetPointCloudFromFile(point_cloud_path, file_p_list, 0)

In [14]:
""" statistical outlier filter """
pcl_cloud = GetPointCloudFromFile(point_cloud_path, file_p_list, 0)

sof_filter = pcl_cloud.make_statistical_outlier_filter()
sof_filter.set_mean_k (10)
sof_filter.set_std_dev_mul_thresh (0.1)
filtered_cloud = sof_filter.filter()

del pcl_cloud
pcl_cloud = filtered_cloud

tree = pcl_cloud.make_kdtree()

ec = pcl_cloud.make_EuclideanClusterExtraction()
ec.set_ClusterTolerance(0.3)
# ec.set_ClusterTolerance(0.5) # --> for world_MotorbikeSimple
# ec.set_ClusterTolerance(0.6) # --> for world_TruckSimple
ec.set_MinClusterSize(50)
# ec.set_MinClusterSize(60) # --> for world_Pedestrian
ec.set_MaxClusterSize(25000)
ec.set_SearchMethod(tree)
cluster_indices = ec.Extract()

clusters = []

for j, indices in enumerate(cluster_indices):

    points = np.zeros((len(indices), 3), dtype=np.float32)
    
    cluster = pcl.PointCloud()

    for i, indice in enumerate(indices):
        points[i,0] = pcl_cloud[indice][0] * -1.
        points[i,1] = pcl_cloud[indice][1]
        points[i,2] = pcl_cloud[indice][2]
               
    clusters.append(points)
    
    print len(points)
    
print "clusters:", len(clusters)

image = cv2.imread(image_path)
cv2.imshow('img', image)
%matplotlib qt

plt.close("all")
display_point_cloud_8_image(clusters, 1.0, image)
        

564
554
271
231
210
204
202
173
clusters: 8


In [21]:
""" Bike """
""" Data arrangement """
%matplotlib qt
plt.close("all")
image = cv2.imread(image_path)
# clusters_to_save = np.array(clusters)[[6, 2, 5, 1, 0, 4, 3, 7]] # world_MotorbikeSimple
clusters_to_save = np.array(clusters)[[3, 2, 7, 0, 1, 6, 5, 4]] # world_ScooterSimple

display_point_cloud_8_image(clusters_to_save, 1.0, image)

In [22]:
""" Bikes """
""" Load and save pickle """

## Load pickle
data_path = os.path.abspath('../dataset/db_v4_20181031.pickle')

with open(data_path, 'rb') as f:
    db = pickle.load(f)
    
db_add_truck = db

db_add_truck[bikes_list[idx]] = []
for cloud in clusters_to_save:
    db_add_truck[bikes_list[idx]].append(cloud)
    
with open('db_v4_20181031.pickle', 'wb') as f:
    pickle.dump(db_add_truck, f)

In [24]:
""" Load pickle and display """

image = cv2.imread(image_path)

# %matplotlib inline
## Load pickle
with open('db_v4_20181031.pickle', 'rb') as f:
    db = pickle.load(f)
%matplotlib qt

# display_point_cloud_8_image(db[cars_list[idx]], 0.1, image)
# for i in range(len(db)):
plt.close('all')
display_point_cloud_8_image(db[bikes_list[1]], 1., image)
print bikes_list[0]

world_MotorbikeSimple
